This notebook performs the analyses to create the raw data (but not plot) Figures 1 and 2 and Supplementary Figures 1-9.

In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import tqdm
from statsmodels.stats.proportion import proportion_confint
import statsmodels.api as sm
from scipy.optimize import minimize_scalar

In [ ]:
maf = pd.read_parquet(
    '/home/jupyter/data/maf/full_maf_merged.parquet'
).drop(
    columns = ['index', 'NCHROBS_afr', 'NCHROBS_eur']
).query(
    'A2.str.len() == 1'
).query(
    'A1.str.len() == 1'
)

In [ ]:
maf_annotated = []
for chrom in tqdm.tqdm(range(1,23)):
    annotations = pd.read_csv(
        f'/home/jupyter/data/annotations/synonymous_and_nonsynonymous/chr{chrom}_afr70346.annot.gz', 
        sep = '\t'
    ).drop(
        columns = ['CM', 'BP']
    )
    maf_annotated.append(maf.query('CHR == @chrom').merge(annotations))
maf_annotated = pd.concat(maf_annotated)

# figure 1a - grid

## f1a,sf2,sf3 - grid

In [ ]:
maf_bin_means_long = maf_annotated.assign(
    eur_maf_bin = lambda df: pd.cut(df.MAF_eur70346, bins = [0, .002, .005, .05, .1, .17, .26, .38, .5], include_lowest = True, right = False),
    afr_maf_bin = lambda df: pd.cut(df.MAF_afr_unadmixed, bins = [0, .002, .005, .05, .1, .17, .26, .38, .5], include_lowest = True, right = False)
).groupby(
    ['eur_maf_bin', 'afr_maf_bin']
)[
    ['synonymous', 'non_synonymous']
].apply(lambda df: pd.DataFrame({
    'non_synonymous' : np.mean(df.non_synonymous),
    'synonymous' : np.mean(df.synonymous),
    'n' : [df.shape[0]],
    'non_synonymous_se' : np.sqrt(np.mean(df.non_synonymous) * (1 - np.mean(df.non_synonymous)) / df.shape[0])
})
).reset_index()
maf_bin_means_long.to_csv('/home/jupyter/data/download/nonsynonymous_eur_afr_maf_props.csv', index = False)

## sf1 - Number of SNPs in each grid bin

In [ ]:
maf_bin_means_long = maf_annotated.assign(
    eur_maf_bin = lambda df: pd.cut(df.MAF_eur70346, bins = [0, .002, .005, .05, .1, .17, .26, .38, .5], include_lowest = True, right = False),
    afr_maf_bin = lambda df: pd.cut(df.MAF_afr_unadmixed, bins = [0, .002, .005, .05, .1, .17, .26, .38, .5], include_lowest = True, right = False)
).groupby(
    ['eur_maf_bin', 'afr_maf_bin']
).apply(lambda df: pd.DataFrame({'n' : [df.shape[0]]})).reset_index()
maf_bin_means_long.sort_values('n')

In [ ]:
maf_bin_means_wide = maf_bin_means_long.pivot_table(
    index = ['afr_maf_bin'], columns = ['eur_maf_bin'], values = 'non_synonymous'
).apply(
    lambda df: df*100
)
maf_bin_means_wide.iloc[0,0] = np.nan
maf_bin_means_wide

In [ ]:
afr_maf_bins_long = maf_annotated.assign(
    eur_maf_bin = lambda df: pd.cut(df.MAF_eur70346, bins = [0, .002, .005, .05, .1, .17, .26, .38, .5], include_lowest = True, right = False),
    afr_maf_bin = lambda df: pd.cut(df.MAF_afr_unadmixed, bins = [0, .002, .005, .05, .1, .17, .26, .38, .5], include_lowest = True, right = False)
).groupby(
    ['afr_maf_bin']
)[
    ['synonymous', 'non_synonymous']
].mean(
).reset_index(
).query(
    'afr_maf_bin != "(0.0, 0.002]"'
)
afr_maf_bins_long.to_csv('/home/jupyter/data/download/nonsynonymous_afr_maf_props.csv')

In [ ]:
eur_maf_bins_long = maf_annotated.assign(
    eur_maf_bin = lambda df: pd.cut(df.MAF_eur70346, bins = [0, .002, .005, .05, .1, .17, .26, .38, .5], include_lowest = True, right = False),
    afr_maf_bin = lambda df: pd.cut(df.MAF_afr_unadmixed, bins = [0, .002, .005, .05, .1, .17, .26, .38, .5], include_lowest = True, right = False)
).groupby(
    ['eur_maf_bin']
)[
    ['synonymous', 'non_synonymous']
].mean(
).reset_index(
).query(
    'eur_maf_bin != "(0.0, 0.002]"'
)
eur_maf_bins_long.to_csv('/home/jupyter/data/download/nonsynonymous_eur_maf_props.csv')

# figures 1b-c - nested deciles

## f1b - Effect of African MAF stratified by European MAF, restricted to p_A > 0.002

In [ ]:
prop_ns_by_afr_maf = maf_annotated.query(
    'MAF_afr_unadmixed >= .002'
).assign(
    afr_maf_bin = lambda df: pd.qcut(df.MAF_afr_unadmixed, 10, duplicates = 'drop').astype(str)
).groupby(
    ['afr_maf_bin']
).apply(lambda df: pd.DataFrame({
    'non_synonymous' : [df.non_synonymous.astype(int).mean()],
    'non_synonymous_lower' : proportion_confint(df.non_synonymous.astype(int).sum(), df.shape[0], method = 'agresti_coull')[0],
    'non_synonymous_upper' : proportion_confint(df.non_synonymous.astype(int).sum(), df.shape[0], method = 'agresti_coull')[1],
    'MAF_afr_unadmixed' : df.MAF_afr_unadmixed.mean()
})).reset_index(
).assign(
    non_synonymous_lower_distance = lambda df: df.non_synonymous - df.non_synonymous_lower,
    non_synonymous_upper_distance = lambda df: df.non_synonymous_upper - df.non_synonymous,
    eur_maf_bin = 'all'
)

prop_ns_by_afr_maf_eur_maf_stratified = maf_annotated.query(
    'MAF_afr_unadmixed >= .002'
).assign(
    eur_maf_bin = lambda df: pd.cut(df.MAF_eur70346, bins = [0, .005, .05, .5], include_lowest = True).astype(str)
).groupby(
    'eur_maf_bin'
).apply(lambda df: 
    df.assign(afr_maf_bin = pd.qcut(df.MAF_afr_unadmixed, 10, duplicates = 'drop').astype(str))
).reset_index(
    drop = True
).groupby(
    ['afr_maf_bin', 'eur_maf_bin']
).apply(lambda df: pd.DataFrame({
    'non_synonymous' : [df.non_synonymous.mean()],
    'non_synonymous_lower' : proportion_confint(df.non_synonymous.sum(), df.shape[0], method = 'agresti_coull')[0],
    'non_synonymous_upper' : proportion_confint(df.non_synonymous.sum(), df.shape[0], method = 'agresti_coull')[1],
    'MAF_afr_unadmixed' : df.MAF_afr_unadmixed.mean()
})).reset_index(
).assign(
    non_synonymous_lower_distance = lambda df: df.non_synonymous - df.non_synonymous_lower,
    non_synonymous_upper_distance = lambda df: df.non_synonymous_upper - df.non_synonymous
)
pd.concat(
    [prop_ns_by_afr_maf, prop_ns_by_afr_maf_eur_maf_stratified]
).sort_values(
    ['eur_maf_bin', 'MAF_afr_unadmixed']
).to_csv('/home/jupyter/data/download/prop_ns_by_afr_maf_eur_maf_stratified.csv', index = False)

px.line(
    pd.concat([prop_ns_by_afr_maf, prop_ns_by_afr_maf_eur_maf_stratified]).sort_values('MAF_afr_unadmixed'),
    x = 'MAF_afr_unadmixed',
    y = 'non_synonymous',
    color = 'eur_maf_bin',
    error_y = 'non_synonymous_upper_distance', 
    error_y_minus = 'non_synonymous_lower_distance',
    markers = True,
    log_x = True,
    log_y = False 
)

In [ ]:
query_list = ['index==index', 'MAF_eur70346 < .005', '.005 <= MAF_eur70346 < .05', '.05 <= MAF_eur70346']
maf_afr_ge_002 = sm.add_constant(
    maf_annotated.query('MAF_afr_unadmixed >= .002'),
    prepend = False
).assign(
    log_maf = lambda df: np.log(df.MAF_afr_unadmixed)
)
afr_res_list = []
for q in tqdm.tqdm(query_list):
    maf_afr_ge_002_queried = maf_afr_ge_002.query(q)
    mod = sm.Logit(
        endog = maf_afr_ge_002_queried.non_synonymous, 
        exog = maf_afr_ge_002_queried[['const', 'log_maf']],
    )
    res = mod.fit(disp = False)
    afr_res_list.append(res)
    
model_prediction_df_list = []
grid = np.log(np.linspace(.002, .5, int(1e4)))
X_df = sm.add_constant(pd.DataFrame({'log_maf' : grid}), prepend = True)
             
for i in range(4):
   pred = afr_res_list[i].get_prediction(
       X_df
   ).summary_frame(
       alpha = .05
   ).rename(
       columns = {'predicted' : 'p_non_synonymous'}
   ).assign(
       log_maf = grid,
       query = query_list[i]
   )
   model_prediction_df_list.append(pred)
model_prediction_df = pd.concat(model_prediction_df_list)
model_prediction_df.to_csv(
   '/home/jupyter/data/download/ns_afr_maf_logistic_predictions.csv',
   index = False
)

## sf5b - Effect of African MAF stratified by European MAF, restricted to p_A> 0.002 and p_E > 0.002

In [ ]:
prop_ns_by_afr_maf_common_domain = maf_annotated.query(
    'MAF_afr_unadmixed >= .002'
).query(
    'MAF_eur70346 >= .002'
).assign(
    afr_maf_bin = lambda df: pd.qcut(df.MAF_afr_unadmixed, 10, duplicates = 'drop').astype(str)
).groupby(
    ['afr_maf_bin']
).apply(lambda df: pd.DataFrame({
    'non_synonymous' : [df.non_synonymous.astype(int).mean()],
    'non_synonymous_lower' : proportion_confint(df.non_synonymous.astype(int).sum(), df.shape[0], method = 'agresti_coull')[0],
    'non_synonymous_upper' : proportion_confint(df.non_synonymous.astype(int).sum(), df.shape[0], method = 'agresti_coull')[1],
    'MAF_afr_unadmixed' : df.MAF_afr_unadmixed.mean()
})).reset_index(
).assign(
    non_synonymous_lower_distance = lambda df: df.non_synonymous - df.non_synonymous_lower,
    non_synonymous_upper_distance = lambda df: df.non_synonymous_upper - df.non_synonymous,
    eur_maf_bin = 'all'
)

prop_ns_by_afr_maf_eur_maf_stratified_common_domain = maf_annotated.query(
    'MAF_afr_unadmixed >= .002'
).query(
    'MAF_eur70346 >= .002'
).assign(
    eur_maf_bin = lambda df: pd.cut(df.MAF_eur70346, bins = [0, .005, .05, .5], include_lowest = True).astype(str)
).groupby(
    'eur_maf_bin'
).apply(lambda df: 
    df.assign(afr_maf_bin = pd.qcut(df.MAF_afr_unadmixed, 10, duplicates = 'drop').astype(str))
).reset_index(
    drop = True
).groupby(
    ['afr_maf_bin', 'eur_maf_bin']
).apply(lambda df: pd.DataFrame({
    'non_synonymous' : [df.non_synonymous.mean()],
    'non_synonymous_lower' : proportion_confint(df.non_synonymous.sum(), df.shape[0], method = 'agresti_coull')[0],
    'non_synonymous_upper' : proportion_confint(df.non_synonymous.sum(), df.shape[0], method = 'agresti_coull')[1],
    'MAF_afr_unadmixed' : df.MAF_afr_unadmixed.mean()
})).reset_index(
).assign(
    non_synonymous_lower_distance = lambda df: df.non_synonymous - df.non_synonymous_lower,
    non_synonymous_upper_distance = lambda df: df.non_synonymous_upper - df.non_synonymous
)
pd.concat(
    [prop_ns_by_afr_maf_common_domain, prop_ns_by_afr_maf_eur_maf_stratified_common_domain]
).sort_values(
    ['eur_maf_bin', 'MAF_afr_unadmixed']
).to_csv('/home/jupyter/data/download/prop_ns_by_afr_maf_eur_maf_stratified_afr_eur_ge_.002.csv', index = False)

#px.line(
#    pd.concat([prop_ns_by_afr_maf_common_domain, prop_ns_by_afr_maf_eur_maf_stratified_common_domain]).sort_values('MAF_afr_unadmixed'),
#    x = 'MAF_afr_unadmixed',
#    y = 'non_synonymous',
#    color = 'eur_maf_bin',
#    error_y = 'non_synonymous_upper_distance', 
#    error_y_minus = 'non_synonymous_lower_distance',
#    markers = True,
#    log_x = True,
#    log_y = False 
#)

In [ ]:
query_list = ['index==index', 'MAF_eur70346 < .005', '.005 <= MAF_eur70346 < .05', '.05 <= MAF_eur70346']
maf_afr_eur_ge_002 = sm.add_constant(
    maf_annotated.query('MAF_afr_unadmixed >= .002').query('MAF_eur70346 >= .002'),
    prepend = False
).assign(
    log_maf = lambda df: np.log(df.MAF_afr_unadmixed)
)
afr_res_list = []
for q in tqdm.tqdm(query_list):
    maf_afr_eur_ge_002_queried = maf_afr_eur_ge_002.query(q)
    mod = sm.Logit(
        endog = maf_afr_eur_ge_002_queried.non_synonymous, 
        exog = maf_afr_eur_ge_002_queried[['const', 'log_maf']],
    )
    res = mod.fit(disp = False)
    afr_res_list.append(res)
    
    
    
model_prediction_df_list = []
grid = np.log(np.linspace(.002, .5, int(1e4)))
X_df = sm.add_constant(pd.DataFrame({'log_maf' : grid}), prepend = True)
              
for i in range(4):
    pred = afr_res_list[i].get_prediction(
        X_df
    ).summary_frame(
        alpha = .05
    ).rename(
        columns = {'predicted' : 'p_non_synonymous'}
    ).assign(
        log_maf = grid,
        query = query_list[i]
    )
    model_prediction_df_list.append(pred)
model_prediction_df = pd.concat(model_prediction_df_list)
model_prediction_df.to_csv(
    '/home/jupyter/data/download/ns_afr_maf_logistic_predictions_afr_eur_ge_.002.csv',
    index = False
)
px.line(
    model_prediction_df,
    y = 'p_non_synonymous', 
    x = 'log_maf', 
    color = 'query',
)   


## f1c - Effect of European MAF stratified by African MAF, restricted to p_E > 0.002

In [ ]:
prop_ns_by_eur_maf = maf_annotated.query(
    'MAF_eur70346 >= .002'
).assign(
    eur_maf_bin = lambda df: pd.qcut(df.MAF_eur70346, 10, duplicates = 'drop').astype(str)
).groupby(
    ['eur_maf_bin']
).apply(lambda df: pd.DataFrame({
    'non_synonymous' : [df.non_synonymous.mean()],
    'non_synonymous_lower' : proportion_confint(df.non_synonymous.sum(), df.shape[0], method = 'agresti_coull')[0],
    'non_synonymous_upper' : proportion_confint(df.non_synonymous.sum(), df.shape[0], method = 'agresti_coull')[1],
    'MAF_eur70346' : df.MAF_eur70346.mean()
})).reset_index(
).assign(
    non_synonymous_lower_distance = lambda df: df.non_synonymous - df.non_synonymous_lower,
    non_synonymous_upper_distance = lambda df: df.non_synonymous_upper - df.non_synonymous,
    afr_maf_bin = 'all'
)

prop_ns_by_eur_maf_afr_maf_stratified = maf_annotated.query(
    'MAF_eur70346 >= .002'
).assign(
    afr_maf_bin = lambda df: pd.cut(df.MAF_afr_unadmixed, bins = [0, .005, .05, .5], include_lowest = True).astype(str)
).groupby(
    'afr_maf_bin'
).apply(lambda df: 
    df.assign(eur_maf_bin = pd.qcut(df.MAF_eur70346, 10, duplicates = 'drop').astype(str))
).reset_index(
    drop = True
).groupby(
    ['afr_maf_bin', 'eur_maf_bin']
).apply(lambda df: pd.DataFrame({
    'non_synonymous' : [df.non_synonymous.mean()],
    'non_synonymous_lower' : proportion_confint(df.non_synonymous.sum(), df.shape[0], method = 'agresti_coull')[0],
    'non_synonymous_upper' : proportion_confint(df.non_synonymous.sum(), df.shape[0], method = 'agresti_coull')[1],
    'MAF_eur70346' : df.MAF_eur70346.mean()
})).reset_index(
).assign(
    non_synonymous_lower_distance = lambda df: df.non_synonymous - df.non_synonymous_lower,
    non_synonymous_upper_distance = lambda df: df.non_synonymous_upper - df.non_synonymous
)

pd.concat(
    [prop_ns_by_eur_maf, prop_ns_by_eur_maf_afr_maf_stratified]
).sort_values(
    ['afr_maf_bin', 'MAF_eur70346']
).to_csv('/home/jupyter/data/download/prop_ns_by_eur_maf_afr_maf_stratified.csv', index = False)

px.line(
    pd.concat([prop_ns_by_eur_maf, prop_ns_by_eur_maf_afr_maf_stratified]).sort_values('MAF_eur70346'),
    x = 'MAF_eur70346',
    y = 'non_synonymous',
    color = 'afr_maf_bin',
    error_y = 'non_synonymous_upper_distance', 
    error_y_minus = 'non_synonymous_lower_distance',
    markers = True,
    log_x = True,
    log_y = True
)

In [ ]:
query_list = ['index==index', 'MAF_afr_unadmixed < .005', '.005 <= MAF_afr_unadmixed < .05', '.05 <= MAF_afr_unadmixed']
maf_eur_ge_002 = sm.add_constant(
    maf_annotated.query('MAF_eur70346 >= .002'),
    prepend = False
).assign(
    log_maf = lambda df: np.log(df.MAF_eur70346)
)
eur_res_list = []
for q in tqdm.tqdm(query_list):
    maf_eur_ge_002_queried = maf_eur_ge_002.query(q)
    mod = sm.Logit(
        endog = maf_eur_ge_002_queried.non_synonymous, 
        exog = maf_eur_ge_002_queried[['const', 'log_maf']],
    )
    res = mod.fit(disp = False)
    eur_res_list.append(res)
    
model_prediction_df_list = []
grid = np.log(np.linspace(.002, .5, int(1e4)))
X_df = sm.add_constant(pd.DataFrame({'log_maf' : grid}), prepend = True)
              
for i in range(4):
    pred = eur_res_list[i].get_prediction(
        X_df
    ).summary_frame(
        alpha = .05
    ).rename(
        columns = {'predicted' : 'p_non_synonymous'}
    ).assign(
        log_maf = grid,
        query = query_list[i]
    )
    model_prediction_df_list.append(pred)
    
#model_prediction_df = pd.concat(model_prediction_df_list)
#model_prediction_df.to_csv(
#    '/home/jupyter/data/download/ns_eur_maf_logistic_predictions.csv',
#    index = False
#)
#px.line(
#    model_prediction_df,
#    y = 'p_non_synonymous', 
#    x = 'log_maf', 
#    color = 'query',
#)

## sf5c - Effect of European MAF stratified by African MAF, restricted to p_E > 0.002 and p_A > 0.002

In [ ]:
prop_ns_by_eur_maf_common_domain = maf_annotated.query(
    'MAF_eur70346 >= .002'
).query(
    'MAF_afr_unadmixed >= .002'
).assign(
    eur_maf_bin = lambda df: pd.qcut(df.MAF_eur70346, 10, duplicates = 'drop').astype(str)
).groupby(
    ['eur_maf_bin']
).apply(lambda df: pd.DataFrame({
    'non_synonymous' : [df.non_synonymous.mean()],
    'non_synonymous_lower' : proportion_confint(df.non_synonymous.sum(), df.shape[0], method = 'agresti_coull')[0],
    'non_synonymous_upper' : proportion_confint(df.non_synonymous.sum(), df.shape[0], method = 'agresti_coull')[1],
    'MAF_eur70346' : df.MAF_eur70346.mean()
})).reset_index(
).assign(
    non_synonymous_lower_distance = lambda df: df.non_synonymous - df.non_synonymous_lower,
    non_synonymous_upper_distance = lambda df: df.non_synonymous_upper - df.non_synonymous,
    afr_maf_bin = 'all'
)

prop_ns_by_eur_maf_afr_maf_stratified_common_domain = maf_annotated.query(
    'MAF_eur70346 >= .002'
).query(
    'MAF_afr_unadmixed >= .002'
).assign(
    afr_maf_bin = lambda df: pd.cut(df.MAF_afr_unadmixed, bins = [0, .005, .05, .5], include_lowest = True).astype(str)
).groupby(
    'afr_maf_bin'
).apply(lambda df: 
    df.assign(eur_maf_bin = pd.qcut(df.MAF_eur70346, 10, duplicates = 'drop').astype(str))
).reset_index(
    drop = True
).groupby(
    ['afr_maf_bin', 'eur_maf_bin']
).apply(lambda df: pd.DataFrame({
    'non_synonymous' : [df.non_synonymous.mean()],
    'non_synonymous_lower' : proportion_confint(df.non_synonymous.sum(), df.shape[0], method = 'agresti_coull')[0],
    'non_synonymous_upper' : proportion_confint(df.non_synonymous.sum(), df.shape[0], method = 'agresti_coull')[1],
    'MAF_eur70346' : df.MAF_eur70346.mean()
})).reset_index(
).assign(
    non_synonymous_lower_distance = lambda df: df.non_synonymous - df.non_synonymous_lower,
    non_synonymous_upper_distance = lambda df: df.non_synonymous_upper - df.non_synonymous
)

pd.concat(
    [prop_ns_by_eur_maf_common_domain, prop_ns_by_eur_maf_afr_maf_stratified_common_domain]
).sort_values(
    ['afr_maf_bin', 'MAF_eur70346']
).to_csv('/home/jupyter/data/download/prop_ns_by_eur_maf_afr_maf_stratified_afr_eur_ge_.002.csv', index = False)

px.line(
    pd.concat([prop_ns_by_eur_maf_common_domain, prop_ns_by_eur_maf_afr_maf_stratified_common_domain]).sort_values('MAF_eur70346'),
    x = 'MAF_eur70346',
    y = 'non_synonymous',
    color = 'afr_maf_bin',
    error_y = 'non_synonymous_upper_distance', 
    error_y_minus = 'non_synonymous_lower_distance',
    markers = True,
    log_x = True,
    log_y = True
)

In [ ]:
query_list = ['index==index', 'MAF_afr_unadmixed < .005', '.005 <= MAF_afr_unadmixed < .05', '.05 <= MAF_afr_unadmixed']
maf_afr_eur_ge_002 = sm.add_constant(
    maf_annotated.query('MAF_eur70346 >= .002').query('MAF_afr_unadmixed >= .002'),
    prepend = False
).assign(
    log_maf = lambda df: np.log(df.MAF_eur70346)
)
eur_res_list = []
for q in tqdm.tqdm(query_list):
    maf_afr_eur_ge_002_queried = maf_afr_eur_ge_002.query(q)
    mod = sm.Logit(
        endog = maf_afr_eur_ge_002_queried.non_synonymous, 
        exog = maf_afr_eur_ge_002_queried[['const', 'log_maf']],
    )
    res = mod.fit(disp = False)
    eur_res_list.append(res)
    
    
model_prediction_df_list = []
grid = np.log(np.linspace(.002, .5, int(1e4)))
X_df = sm.add_constant(pd.DataFrame({'log_maf' : grid}), prepend = True)
              
for i in range(4):
    pred = eur_res_list[i].get_prediction(
        X_df
    ).summary_frame(
        alpha = .05
    ).rename(
        columns = {'predicted' : 'p_non_synonymous'}
    ).assign(
        log_maf = grid,
        query = query_list[i]
    )
    model_prediction_df_list.append(pred)
    
model_prediction_df = pd.concat(model_prediction_df_list)
model_prediction_df.to_csv(
    '/home/jupyter/data/download/ns_eur_maf_logistic_predictions_afr_eur_ge_.002.csv',
    index = False
)
px.line(
    model_prediction_df,
    y = 'p_non_synonymous', 
    x = 'log_maf', 
    color = 'query',
    log_y = True
)

# Figure 2 - non-synonymous ~ p_mix

In [ ]:
# ASSUMES M1 AND M2 HAVE THE SAME NUMBER OF DOF
from scipy.stats import norm
def vuong_test(m1_res, m1_mod, m2_res, m2_mod):
    m1_pointwise_ll = m1_mod.loglikeobs(m1_res.params)
    m2_pointwise_ll = m2_mod.loglikeobs(m2_res.params)
    pointwise_llr = m1_pointwise_ll - m2_pointwise_ll
    llr = np.sum(pointwise_llr)
    omega2 = np.var(pointwise_llr, ddof=1)
    z = llr / np.sqrt(pointwise_llr.shape[0] * omega2)
    p = (1 - norm.cdf(np.abs(z))) * 2
    return z, p



## f2 - Including variants with MAF > .002 in either, and thresholding p

In [ ]:
maf_ge_002_in_either = sm.add_constant(
    maf_annotated.query('(MAF_afr_unadmixed >= .002) | (MAF_eur70346 >= .002)'),
    prepend = False
).assign(
    MAF_afr_unadmixed = lambda df: np.maximum(df.MAF_afr_unadmixed, .002),
    MAF_eur70346 = lambda df: np.maximum(df.MAF_eur70346, .002)
)
def pmix_fit(w):
    log_p_mix = np.log(w * maf_ge_002_in_either.MAF_afr_unadmixed + (1 - w) * maf_ge_002_in_either.MAF_eur70346)
    mod = sm.Logit(
        endog = maf_ge_002_in_either.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : log_p_mix})
    )
    return mod.fit(disp = False)

def pmix_profile_l(w):
    return pmix_fit(w).llf
    
def pmix_negative_profile_l(w):
    return -1 * pmix_profile_l(w)

In [ ]:
w_grid = np.linspace(0, 1, 101)
res_grid = [pmix_fit(w) for w in tqdm.tqdm(w_grid)]

In [ ]:
l = [r.llf for r in res_grid]
beta_se = [r.bse['p_mix'] for r in res_grid]
beta = [r.params['p_mix'] for r in res_grid]
grid_fits = pd.dataframe({
    'w' : w_grid,
    'l' : l,
    'beta' : beta, 
    'beta_se' : beta_se
})
grid_fits.to_csv('/home/jupyter/data/p_mix_results/ns_p_mix_params_ge.002_in_either_afr_eur.csv')

In [ ]:
w_mle = minimize_scalar(pmix_negative_profile_l, bounds = [0, 1], options={'disp': True} )
mle_l = pmix_profile_l(w_mle.x)

def distance_from_ci_bound(w):
    l = pmix_profile_l(w)
    return np.abs(mle_l - l - 1.92)

w_ci_lower = minimize_scalar(distance_from_ci_bound, bounds = [0, w_mle.x], options={'disp': True} )
w_ci_upper = minimize_scalar(distance_from_ci_bound, bounds = [w_mle.x, 1], options={'disp': True} )

In [ ]:
maf_ge_002_in_either = sm.add_constant(
    maf_annotated.query('(MAF_afr_unadmixed >= .002) | (MAF_eur70346 >= .002)'),
    prepend = False
).assign(
    MAF_afr_unadmixed = lambda df: np.maximum(df.MAF_afr_unadmixed, .002),
    MAF_eur70346 = lambda df: np.maximum(df.MAF_eur70346, .002)
)
def pmix_fit(w):
    log_p_mix = np.log(w * maf_ge_002_in_either.MAF_afr_unadmixed + (1 - w) * maf_ge_002_in_either.MAF_eur70346)
    mod = sm.Logit(
        endog = maf_ge_002_in_either.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : log_p_mix})
    )
    return mod.fit(disp = False), mod

eur_res, eur_mod = pmix_fit(0)
afr_res, afr_mod = pmix_fit(1)
vuong_test(eur_res, eur_mod, afr_res, afr_mod)

## sf6,8 - restricting to variants with MAF > .002 in Eur

In [ ]:
maf_ge_002_in_eur = sm.add_constant(
    maf_annotated.query('(MAF_eur70346 >= .002)'),
    prepend = False
).assign(
    MAF_afr_unadmixed = lambda df: np.maximum(df.MAF_afr_unadmixed, .002),
    MAF_eur70346 = lambda df: np.maximum(df.MAF_eur70346, .002)
)
def pmix_fit(w):
    log_p_mix = np.log(w * maf_ge_002_in_eur.MAF_afr_unadmixed + (1 - w) * maf_ge_002_in_eur.MAF_eur70346)
    mod = sm.Logit(
        endog = maf_ge_002_in_eur.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : log_p_mix})
    )
    return mod.fit(disp = False)

def pmix_profile_l(w):
    return pmix_fit(w).llf
    
def pmix_negative_profile_l(w):
    return -1 * pmix_profile_l(w)

In [ ]:
w_grid = np.linspace(0, 1, 101)
res_grid = [pmix_fit(w) for w in tqdm.tqdm(w_grid)]

In [ ]:
l = [r.llf for r in res_grid]
beta_se = [r.bse['p_mix'] for r in res_grid]
beta = [r.params['p_mix'] for r in res_grid]
grid_fits = pd.DataFrame({
    'w' : w_grid,
    'l' : l,
    'beta' : beta, 
    'beta_se' : beta_se
})
grid_fits.to_csv('/home/jupyter/data/p_mix_results/ns_p_mix_params_ge.002_in_eur.csv')

In [ ]:
w_mle = minimize_scalar(pmix_negative_profile_l, bounds = [0, 1], options={'disp': True} )
mle_l = pmix_profile_l(w_mle.x)

def distance_from_ci_bound(w):
    l = pmix_profile_l(w)
    return np.abs(mle_l - l - 1.92)

w_ci_lower = minimize_scalar(distance_from_ci_bound, bounds = [0, w_mle.x], options={'disp': True} )
w_ci_upper = minimize_scalar(distance_from_ci_bound, bounds = [w_mle.x, 1], options={'disp': True} )

In [ ]:
maf_ge_002_in_eur = sm.add_constant(
    maf_annotated.query('(MAF_eur70346 >= .002)'),
    prepend = False
).assign(
    MAF_afr_unadmixed = lambda df: np.maximum(df.MAF_afr_unadmixed, .002),
    MAF_eur70346 = lambda df: np.maximum(df.MAF_eur70346, .002)
)
def pmix_fit(w):
    log_p_mix = np.log(w * maf_ge_002_in_eur.MAF_afr_unadmixed + (1 - w) * maf_ge_002_in_eur.MAF_eur70346)
    mod = sm.Logit(
        endog = maf_ge_002_in_eur.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : log_p_mix})
    )
    return mod.fit(disp = False), mod

eur_res, eur_mod = pmix_fit(0)
afr_res, afr_mod = pmix_fit(1)
vuong_test(eur_res, eur_mod, afr_res, afr_mod)

## sf6,8 - restricting to variants with MAF > 0.002 in afr

In [ ]:
maf_ge_002_in_afr = sm.add_constant(
    maf_annotated.query('(MAF_afr_unadmixed >= .002)'),
    prepend = False
).assign(
    MAF_afr_unadmixed = lambda df: np.maximum(df.MAF_afr_unadmixed, .002),
    MAF_eur70346 = lambda df: np.maximum(df.MAF_eur70346, .002)
)
def pmix_fit(w):
    log_p_mix = np.log(w * maf_ge_002_in_afr.MAF_afr_unadmixed + (1 - w) * maf_ge_002_in_afr.MAF_eur70346)
    mod = sm.Logit(
        endog = maf_ge_002_in_afr.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : log_p_mix})
    )
    return mod.fit(disp = False)

def pmix_profile_l(w):
    return pmix_fit(w).llf
    
def pmix_negative_profile_l(w):
    return -1 * pmix_profile_l(w)

In [ ]:
w_grid = np.linspace(0, 1, 101)
res_grid = [pmix_fit(w) for w in tqdm.tqdm(w_grid)]

In [ ]:
l = [r.llf for r in res_grid]
beta_se = [r.bse['p_mix'] for r in res_grid]
beta = [r.params['p_mix'] for r in res_grid]
grid_fits = pd.DataFrame({
    'w' : w_grid,
    'l' : l,
    'beta' : beta, 
    'beta_se' : beta_se
})
grid_fits.to_csv('/home/jupyter/data/p_mix_results/ns_p_mix_params_ge.002_in_afr.csv')

In [ ]:
w_mle = minimize_scalar(pmix_negative_profile_l, bounds = [0, 1], options={'disp': True} )
mle_l = pmix_profile_l(w_mle.x)

def distance_from_ci_bound(w):
    l = pmix_profile_l(w)
    return np.abs(mle_l - l - 1.92)

w_ci_lower = minimize_scalar(distance_from_ci_bound, bounds = [0, w_mle.x], options={'disp': True} )
w_ci_upper = minimize_scalar(distance_from_ci_bound, bounds = [w_mle.x, 1], options={'disp': True} )

In [ ]:
maf_ge_002_in_afr = sm.add_constant(
    maf_annotated.query('(MAF_afr_unadmixed >= .002)'),
    prepend = False
).assign(
    MAF_afr_unadmixed = lambda df: np.maximum(df.MAF_afr_unadmixed, .002),
    MAF_eur70346 = lambda df: np.maximum(df.MAF_eur70346, .002)
)
def pmix_fit(w):
    log_p_mix = np.log(w * maf_ge_002_in_afr.MAF_afr_unadmixed + (1 - w) * maf_ge_002_in_afr.MAF_eur70346)
    mod = sm.Logit(
        endog = maf_ge_002_in_afr.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : log_p_mix})
    )
    return mod.fit(disp = False), mod

eur_res, eur_mod = pmix_fit(0)
afr_res, afr_mod = pmix_fit(1)
vuong_test(eur_res, eur_mod, afr_res, afr_mod)

## sf6,8 - restricting to variants with MAF > .002 in both

In [ ]:
maf_ge_002_in_both = sm.add_constant(
    maf_annotated.query('MAF_afr_unadmixed >= .002').query('MAF_eur70346 >= .002'),
    prepend = False
)
def pmix_fit(w):
    log_p_mix = np.log(w * maf_ge_002_in_both.MAF_afr_unadmixed + (1 - w) * maf_ge_002_in_both.MAF_eur70346)
    mod = sm.Logit(
        endog = maf_ge_002_in_both.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : log_p_mix})
    )
    return mod.fit(disp = False)

def pmix_profile_l(w):
    return pmix_fit(w).llf
    
def pmix_negative_profile_l(w):
    return -1 * pmix_profile_l(w)
    
#from scipy.optimize import minimize_scalar
#
#w_mle = minimize_scalar(pmix_negative_profile_l, bounds = [0, 1], options={'disp': True} )
#afr_res_list = []
#
#mle_l = pmix_profile_l(w_mle.x)
#def distance_from_ci_bound(w):
#    l = pmix_profile_l(w)
#    return np.abs(mle_l - l - 1.92)
#w_ci_lower = minimize_scalar(distance_from_ci_bound, bounds = [0, w_mle.x], options={'disp': True} )
#w_ci_upper = minimize_scalar(distance_from_ci_bound, bounds = [w_mle.x, 1], options={'disp': True} )
#

In [ ]:
w_grid = np.linspace(0, 1, 101)
res_grid = [pmix_fit(w) for w in tqdm.tqdm(w_grid)]

In [ ]:
l = [r.llf for r in res_grid]
beta_se = [r.bse['p_mix'] for r in res_grid]
beta = [r.params['p_mix'] for r in res_grid]
grid_fits = pd.DataFrame({
    'w' : w_grid,
    'l' : l,
    'beta' : beta, 
    'beta_se' : beta_se
})
grid_fits.to_csv('/home/jupyter/data/p_mix_results/ns_p_mix_params_ge.002_afr_eur.csv')

In [ ]:
grid_fits = pd.read_csv('/home/jupyter/data/p_mix_results/ns_p_mix_params_ge.002_afr_eur.csv')

In [ ]:
maf_ge_002_in_both = sm.add_constant(
    maf_annotated.query('MAF_afr_unadmixed >= .002').query('MAF_eur70346 >= .002'),
    prepend = False
)
def pmix_fit(w):
    log_p_mix = np.log(w * maf_ge_002_in_both.MAF_afr_unadmixed + (1 - w) * maf_ge_002_in_both.MAF_eur70346)
    mod = sm.Logit(
        endog = maf_ge_002_in_both.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : log_p_mix})
    )
    return mod.fit(disp = False), mod

eur_res, eur_mod = pmix_fit(0)
afr_res, afr_mod = pmix_fit(1)
vuong_test(eur_res, eur_mod, afr_res, afr_mod)

## sf6,8 - restricting to variants with MAF > 0.05 in both

In [ ]:
maf_ge_05_in_both = sm.add_constant(
    maf_annotated.query('MAF_afr_unadmixed >= .05').query('MAF_eur70346 >= .05'),
    prepend = False
)
def pmix_fit(w):
    log_p_mix = np.log(w * maf_ge_05_in_both.MAF_afr_unadmixed + (1 - w) * maf_ge_05_in_both.MAF_eur70346)
    mod = sm.Logit(
        endog = maf_ge_05_in_both.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : log_p_mix})
    )
    return mod.fit(disp = False)

def pmix_profile_l(w):
    return pmix_fit(w).llf
    
def pmix_negative_profile_l(w):
    return -1 * pmix_profile_l(w)
    
from scipy.optimize import minimize_scalar

w_mle = minimize_scalar(pmix_negative_profile_l, bounds = [0, 1], options={'disp': True} )
afr_res_list = []

mle_l = pmix_profile_l(w_mle.x)
def distance_from_ci_bound(w):
    l = pmix_profile_l(w)
    return np.abs(mle_l - l - 1.92)
w_ci_lower = minimize_scalar(distance_from_ci_bound, bounds = [0, w_mle.x], options={'disp': True} )
w_ci_upper = minimize_scalar(distance_from_ci_bound, bounds = [w_mle.x, 1], options={'disp': True} )


In [ ]:
w_grid = np.linspace(0, 1, 101)
res_grid = [pmix_fit(w) for w in tqdm.tqdm(w_grid)]

In [ ]:
l = [r.llf for r in res_grid]
beta_se = [r.bse['p_mix'] for r in res_grid]
beta = [r.params['p_mix'] for r in res_grid]
grid_fits = pd.DataFrame({
    'w' : w_grid,
    'l' : l,
    'beta' : beta, 
    'beta_se' : beta_se
})
grid_fits.to_csv('/home/jupyter/data/p_mix_results/ns_p_mix_params_ge.05_afr_eur.csv')

In [ ]:
maf_ge_05_in_both = sm.add_constant(
    maf_annotated.query('MAF_afr_unadmixed >= .05').query('MAF_eur70346 >= .05'),
    prepend = False
)
def pmix_fit(w):
    log_p_mix = np.log(w * maf_ge_05_in_both.MAF_afr_unadmixed + (1 - w) * maf_ge_05_in_both.MAF_eur70346)
    mod = sm.Logit(
        endog = maf_ge_05_in_both.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : log_p_mix})
    )
    return mod.fit(disp = False), mod

def pmix_profile_l(w):
    return pmix_fit(w).llf
    
def pmix_negative_profile_l(w):
    return -1 * pmix_profile_l(w)

eur_res, eur_mod = pmix_fit(0)
afr_res, afr_mod = pmix_fit(1)


## sf9 - Including variants with MAF > .002 in either changing threshold 

In [ ]:
maf_ge_002_in_either = sm.add_constant(
    maf_annotated.query('(MAF_afr_unadmixed >= .002) | (MAF_eur70346 >= .002)'),
    prepend = False
).assign(
    MAF_afr_unadmixed = lambda df: np.maximum(df.MAF_afr_unadmixed, .05),
    MAF_eur70346 = lambda df: np.maximum(df.MAF_eur70346, .05)
)
def pmix_fit(w):
    log_p_mix = np.log(w * maf_ge_002_in_either.MAF_afr_unadmixed + (1 - w) * maf_ge_002_in_either.MAF_eur70346)
    mod = sm.Logit(
        endog = maf_ge_002_in_either.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : log_p_mix})
    )
    return mod.fit(disp = False)

def pmix_profile_l(w):
    return pmix_fit(w).llf
    
def pmix_negative_profile_l(w):
    return -1 * pmix_profile_l(w)

In [ ]:
w_mle = minimize_scalar(pmix_negative_profile_l, bounds = [0, 1], options={'disp': True} )
mle_l = pmix_profile_l(w_mle.x)

def distance_from_ci_bound(w):
    l = pmix_profile_l(w)
    return np.abs(mle_l - l - 1.92)

w_ci_lower = minimize_scalar(distance_from_ci_bound, bounds = [0, w_mle.x], options={'disp': True} )
w_ci_upper = minimize_scalar(distance_from_ci_bound, bounds = [w_mle.x, 1], options={'disp': True} )

In [ ]:
#w_grid = np.linspace(0, 1, 101)
#res_grid = [pmix_fit(w) for w in tqdm.tqdm(w_grid)]
#l = [r.llf for r in res_grid]
#beta_se = [r.bse['p_mix'] for r in res_grid]
#beta = [r.params['p_mix'] for r in res_grid]
#grid_fits = pd.DataFrame({
#    'w' : w_grid,
#    'l' : l,
#    'beta' : beta, 
#    'beta_se' : beta_se
#})
grid_fits.to_csv('/home/jupyter/data/p_mix_results/ns_p_mix_params_ge.002_in_either_afr_eur_05_thresholded.csv')

In [ ]:
w_mle

In [ ]:
w_ci_lower

In [ ]:
w_ci_upper

## sf7 - Compare marginal p_a in African variants to marginal p_E in European variants

In [ ]:
maf_annotated_with_blocks = maf_annotated.assign(
    block = lambda df: (np.arange(df.shape[0]) / (df.shape[0]/200)).astype(int)
)

In [ ]:
maf_ge_002_in_afr = sm.add_constant(
    maf_annotated_with_blocks.query('MAF_afr_unadmixed >= .002'),
    prepend = False
)
maf_ge_002_in_eur = sm.add_constant(
    maf_annotated_with_blocks.query('MAF_eur70346 >= .002'),
    prepend = False
)
eur_r2_loo = []
afr_r2_loo = []
eur_beta_loo = []
afr_beta_loo = []
block_df = pd.DataFrame()
for block in tqdm.tqdm(range(200)):
    maf_ge_002_in_afr_loo = maf_ge_002_in_afr.query('block != @block')
    maf_ge_002_in_eur_loo = maf_ge_002_in_eur.query('block != @block')
    eur_fit = sm.Logit(
        endog = maf_ge_002_in_eur_loo.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : np.log(maf_ge_002_in_eur_loo.MAF_eur70346)})
    ).fit(disp = False)
    afr_fit = sm.Logit(
        endog = maf_ge_002_in_afr_loo.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : np.log(maf_ge_002_in_afr_loo.MAF_afr70346)})
    ).fit(disp = False)
    new_block_df = pd.DataFrame({
        'block' : [block],
        'eur_r2' : eur_fit.prsquared,
        'afr_r2' : afr_fit.prsquared,
        'eur_beta' : eur_fit.params[1],
        'afr_beta' : afr_fit.params[1]
    })
    block_df = pd.concat([block_df, new_block_df])
    block_df.to_csv('/home/jupyter/r2_comparison_blocks.csv', index = False)


In [ ]:
import sys
sys.path.append('/home/jupyter/scripts/multisusie_private/scripts/')
import genetics_utils
r2_jk_df = pd.read_csv(
    '/home/jupyter/r2_comparison_blocks.csv'
).assign(
    delta_r2 = lambda df: df.eur_r2 - df.afr_r2,
    delta_beta = lambda df: df.eur_beta - df.afr_beta
)
jk_sd = np.sqrt(genetics_utils.get_jackknife_variance(r2_jk_df.delta_r2))
jk_mean = r2_jk_df.delta_r2.mean()
jk_mean / jk_sd
jk_sd = np.sqrt(genetics_utils.get_jackknife_variance(r2_jk_df.delta_beta))
jk_mean = r2_jk_df.delta_beta.mean()
jk_mean / jk_sd

In [ ]:
!cat /home/jupyter/scripts/scaled_pmix_estimate.py

## sf4 - Does min beta align with MLE if we rescale pmix?

In [ ]:
maf_ge_002_in_either = sm.add_constant(
    maf_annotated.query('(MAF_afr_unadmixed >= .002) | (MAF_eur70346 >= .002)'),
    prepend = False
).assign(
    MAF_afr_unadmixed = lambda df: np.maximum(df.MAF_afr_unadmixed, .002),
    MAF_eur70346 = lambda df: np.maximum(df.MAF_eur70346, .002)
)
p_mix_var_list = []
for w in np.linspace(0, 1, 101):
    p_mix_var = np.var(np.log(w * maf_annotated.MAF_afr70346 + (1-w) * maf_annotated.MAF_eur70346))
    p_mix_var_list.append(p_mix_var)

In [ ]:
px.line(
    x = np.linspace(0, 1, 101),
    y = p_mix_var_list
)

In [ ]:
maf_ge_002_in_either = sm.add_constant(
    maf_annotated.query('(MAF_afr_unadmixed >= .002) | (MAF_eur70346 >= .002)'),
    prepend = False
).assign(
    MAF_afr_unadmixed = lambda df: np.maximum(df.MAF_afr_unadmixed, .002),
    MAF_eur70346 = lambda df: np.maximum(df.MAF_eur70346, .002)
)
def pmix_fit(w):
    log_p_mix = np.log(w * maf_ge_002_in_either.MAF_afr_unadmixed + (1 - w) * maf_ge_002_in_either.MAF_eur70346)
    log_p_mix = log_p_mix / np.std(log_p_mix)
    mod = sm.Logit(
        endog = maf_ge_002_in_either.non_synonymous,
        exog = pd.DataFrame({'const' : 1, 'p_mix' : log_p_mix})
    )
    return mod.fit(disp = False)

def pmix_profile_l(w):
    return pmix_fit(w).llf
    
def pmix_negative_profile_l(w):
    return -1 * pmix_profile_l(w)

#w_grid = np.linspace(0, 1, 101)
#res_grid = [pmix_fit(w) for w in tqdm.tqdm(w_grid)]

In [ ]:
beta = [r.params['p_mix'] for r in res_grid]
px.line(
    x = w_grid,
    y = beta
)

In [ ]:
w_mle = minimize_scalar(pmix_negative_profile_l, bounds = [0, 1], options={'disp': True} )
mle_l = pmix_profile_l(w_mle.x)